In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 5)

In [3]:
URI = "neo4j://localhost:7999"
AUTH = ("neo4j", "password")

In [4]:
dataset = pd.read_csv("./BestBooksEverClean.csv")
dataset

,title,series,author,genres,pages,publishDate,publishYear,rating,likedPercent,price
0,Ballet Shoes,Shoes,Noel Streatfeild,"['Childrens', 'Classics', 'Fiction', 'Middle G...",233,September 2003,2003,4.06,94.0,3.47
1,An Appetite for Violets,standalone,Martine Bailey,"['Historical Fiction', 'Mystery', 'Fiction', '...",400,January 13th 2015,2015,3.71,91.0,3.99
...,...,...,...,...,...,...,...,...,...,...
31797,Fault Lines: How Hidden Fractures Still Threat...,standalone,Raghuram G. Rajan,"['Economics', 'Nonfiction', 'Finance', 'Busine...",260,May 1st 2010,2010,4.11,95.0,2.55
31798,Lady of Hay,standalone,Barbara Erskine,"['Historical Fiction', 'Historical', 'Fiction'...",600,March 13th 2001,2001,4.05,91.0,3.08


#### From String list, return the list of genres

In [5]:
def genres_to_list(genres: str) -> list:
  list_of_genres = genres.removeprefix('[').removesuffix(']').split(', ')
  return list(map(lambda s: s[1:][:-1], list_of_genres))

In [6]:
dataset["genres"] = dataset["genres"].apply(genres_to_list)
type(dataset['genres'].iloc[0])

list

#### Enable Connection

In [7]:
driver = GraphDatabase.driver(URI, auth=AUTH)
session = driver.session()

In [14]:
def insert_user(name: str) -> bool:
  # Insertar usuario
  result = session.run(f"MATCH (u:User) WHERE u.name = $name RETURN u.name as name", {"name": name})
  if len(result.data()) == 0:
    try:
      session.run("CREATE (u:User {name: $name})", {"name": name})
      return True
    except:
      pass
  return False

def show_users():
  # Mostrar usuarios
  print("Usuarios:")
  result = session.run("MATCH (u:User) RETURN u.name AS name")
  for record in result:
    print(f" - {record["name"]}")

def show_connections():
  # Mostrar vinculos
  print("\nVinculos:")
  result = session.run("MATCH (u:User)-[r:FRIEND]->(u2:User) RETURN u.name AS name, u2.name AS name2")
  for record in result:
    print(f" - {record['name']} es amigo de {record['name2']}")

def insert_author(author: str) -> bool:
  # Insertar autor
  result = session.run("MATCH (a:Author) WHERE a.name = $name RETURN a.name as name", {"name": author})
  if len(result.data()) == 0:
    try:
      session.run("CREATE (a:Author {name: $name})", {"name": author})
      return True
    except:
      pass
  return False

def show_authors():
  # Mostrar autores
  print("Autores:")
  result = session.run("MATCH (a:Author) RETURN a.name AS name")
  for record in result:
    print(f" - {record["name"]}")

def insert_genre(genre: str) -> bool:
  # Insertar Genero
  result = session.run("MATCH (g:Genre) WHERE g.name = $name RETURN g.name as name", {"name": genre})
  if len(result.data()) == 0:
    try:
      session.run("CREATE (g:Genre {name: $name})", {"name": genre})
      return True
    except:
      pass
  return False

def show_genres():
  # Mostrar generos
  print("Generos:")
  result = session.run("MATCH (g:Genre) RETURN g.name AS name")
  for record in result:
    print(f" - {record["name"]}")

def insert_books(books: pd.DataFrame) -> bool:
  # Insertar libros
  query = """
    UNWIND $books AS book
    MERGE (b:Book {title: book.title})
    ON CREATE SET b.pages = book.pages, b.rating = book.rating, b.likedPercent = book.likedPercent, 
                  b.numRating = book.numRating, b.price = book.price, b.publishDate = book.publishDate, 
                  b.publishYear = book.publishYear
    """
  
  try:
    books_dict = books.to_dict('records')
    session.run(query, {"books": books_dict})
    return True
  except:
    pass
  return False

def get_random_books(n: int):
  query = f"""
    MATCH (b:Book)
    RETURN b.title AS title, b.author AS author, b.genres AS genres, b.pages AS pages,
           b.rating AS rating, b.likedPercent AS likedPercent, b.numRating AS numRating,
           b.price AS price, b.publishDate AS publishDate, b.publishYear AS publishYear
    ORDER BY rand()
    LIMIT {n}
    """
  result = session.run(query)
  books = result.data()
  return books

#### Insert Users

In [9]:
users = [
  'Victor',
  'Badre',
  'Daniel',
  'Adrian',
  'Manuel',
  'Angel',
  'Felix',
  'Fernando'
]

In [10]:
for user in users:
  insert_user(user)

In [66]:
show_users()

Usuarios:
 - Victor
 - Badre
 - Daniel
 - Adrian
 - Manuel
 - Angel
 - Felix
 - Fernando


#### Insert authors

In [11]:
for author in dataset['author'].unique():
  insert_author(author)

In [57]:
show_authors()

Autores:
 - Noel Streatfeild
 - Martine Bailey
 - Jeffrey Moore
 - Greg Smith
 - Anthony Burgess
 - Rachel Hawkins
 - Idries Shah
 - Cynthia Eden
 - Jenny Lawson
 - Nancy A. Collins
 - Antonia Fraser
 - Mark Crilley
 - John Updike
 - Lisi Harrison
 - Bell Hooks
 - Bryan Davis
 - Watty Piper
 - Alice Miller
 - Jaswant Singh
 - Eoin Colfer
 - Erin McCarthy
 - Theo Richmond
 - David Almond
 - Karen Traviss
 - Karl Schroeder
 - Tracy J. Butler
 - Brian Jacques
 - Jude Watson
 - Alex Flinn
 - J.G. Ballard
 - Allen Ginsberg
 - Frederick Reiken
 - Gabrielle Tozer
 - Anonymous
 - Jane Bowles
 - Gene Baur
 - Owen Mullen
 - Mike Shevdon
 - Rosanne Rivers
 - Jennifer Niven
 - Carol Gorman
 - Evelyn M. Monahan
 - Andrew C. Skinner
 - Leonardo Sciascia
 - Jane T. Sibley
 - Pepper Winters
 - Tim Bowler
 - K.L. Armstrong
 - William Lane Craig
 - Kelley St. John
 - Hugh MacLeod
 - Markus Zusak
 - Kat Falls
 - Georgette Heyer
 - Jim Grimsley
 - Diana Gabaldon
 - Norah Lofts
 - Suzanne Woods Fisher
 - F

#### Insert Genres

In [13]:
set_of_genres = set()
for genres in dataset['genres'].values:
  for genre in genres:
    set_of_genres.add(genre)
set_of_genres

{'10th Century',
 '11th Century',
 '12th Century',
 '13th Century',
 '14th Century',
 '15th Century',
 '16th Century',
 '17th Century',
 '18th Century',
 '19th Century',
 '1st Grade',
 '20th Century',
 '21st Century',
 '2nd Grade',
 '40k',
 'Abuse',
 'Academia',
 'Academic',
 'Academics',
 'Action',
 'Activism',
 'Adoption',
 'Adult',
 'Adult Fiction',
 'Adventure',
 'Aeroplanes',
 'Africa',
 'African American',
 'African American Literature',
 'African American Romance',
 'African Literature',
 'Agriculture',
 'Airships',
 'Albanian Literature',
 'Alchemy',
 'Alcohol',
 'Algebra',
 'Algeria',
 'Algorithms',
 'Aliens',
 'Alternate History',
 'Alternate Universe',
 'Alternative Medicine',
 'Amateur Sleuth',
 'Amazon',
 'American',
 'American Civil War',
 'American Classics',
 'American Fiction',
 'American History',
 'American Revolution',
 'American Revolutionary War',
 'Americana',
 'Amish',
 'Amish Fiction',
 'Anarchism',
 'Ancient',
 'Ancient History',
 'Angels',
 'Anglo Saxon',
 'A

In [16]:
for genre in set_of_genres:
  insert_genre(genre)

In [17]:
show_genres()

Generos:
 - Cookbooks
 - Danish
 - Crime
 - Libya
 - Survival
 - Spain
 - Black Literature
 - Biblical Fiction
 - Wine
 - Drama
 - Alcohol
 - Turkish Literature
 - Marathi
 - Food and Drink
 - Banks
 - China
 - American Revolution
 - 40k
 - Lebanon
 - Spanish Literature
 - Novels
 - Dragons
 - Urban Planning
 - Buisness
 - Architecture
 - Yuri
 - 10th Century
 - Choose Your Own Adventure
 - Werewolves
 - Terrorism
 - Japanese Literature
 - Book Club
 - 19th Century
 - Musicians
 - Romantic
 - Paranormal
 - Mathematics
 - Womens Fiction
 - Role Playing Games
 - Love
 - Human Development
 - Anti Intellectualism
 - Fiction
 - Recreation
 - Film
 - Social Media
 - Western Romance
 - Harlequin Desire
 - Military Romance
 - Americana
 - Fat Studies
 - Wizards
 - Espionage
 - Diary
 - Fostering
 - Finnish Literature
 - Brain
 - Harlequin Teen
 - Own
 - Virtual Reality
 - Mali
 - London Underground
 - Angels
 - Ornithology
 - Theory
 - Jazz
 - Harlequin
 - Pseudoscience
 - Young Adult Historic

#### Insert books

In [18]:
insert_books(dataset)

True

In [19]:
books = get_random_books(5)
books

[{'title': 'Girls Acting Catty',
  'author': None,
  'genres': None,
  'pages': 179,
  'rating': 3.93,
  'likedPercent': 89.0,
  'numRating': None,
  'price': 3.12,
  'publishDate': 'November 1st 2009',
  'publishYear': 2009},
 {'title': 'Day by Day Armageddon',
  'author': None,
  'genres': None,
  'pages': 260,
  'rating': 4.01,
  'likedPercent': 92.0,
  'numRating': None,
  'price': 1.46,
  'publishDate': 'June 11th 2004',
  'publishYear': 2004},
 {'title': 'Cherry Crush',
  'author': None,
  'genres': None,
  'pages': 272,
  'rating': 4.01,
  'likedPercent': 93.0,
  'numRating': None,
  'price': 2.78,
  'publishDate': 'September 2nd 2010',
  'publishYear': 2010},
 {'title': 'Dragonheart',
  'author': None,
  'genres': None,
  'pages': 538,
  'rating': 4.15,
  'likedPercent': 94.0,
  'numRating': None,
  'price': 5.86,
  'publishDate': 'October 1st 2008',
  'publishYear': 2008},
 {'title': 'Hilda and the Troll',
  'author': None,
  'genres': None,
  'pages': 40,
  'rating': 4.25,
  

#### Close Connection

In [ ]:
session.close()
driver.close()